In [4]:
import pandas as pd
import numpy as np
import statistics
import requests
import json

In [13]:
def callApi(method, stock, isSeriese = False):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last5 = "/5y"
    last10 = "/10"
    period = "period=annual"
    if method == "dividends":
        url = host + ticker + api_method + last5 + query + key
    elif (method == "financials" or method == "balance-sheet") and isSeriese :
        url = host + ticker + api_method + last10 + query + period + add + key
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [33]:
def avgDividends(ticker):
    dividends = callApi("dividends", ticker)
    avgDividend = np.array([])
    print(dividends)
    for dividend in dividends:
        print(dividend)
        if "amount" in dividend.keys() and dividend["amount"] != '': 
            if dividend["amount"] is not None:
                avgDividend = np.append(avgDividend, dividend["amount"])
    if len(avgDividend): 
        return statistics.mean(avgDividend) 
    else:  
        return 0 

In [22]:
def sharesOutstanding(ticker):
    stats = callApi("stats", ticker)
    if not("sharesOutstanding" in stats.keys()):
        return 0.0
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    else:
        return 0.0

In [16]:
def futureValue(bookValue, period, rate ):
    return bookValue * ((1 + rate) ** period )

In [17]:
def bookValues(ticker):
    balancesheets = callApi("balance-sheet", ticker, True)
    balancesheet = callApi("balance-sheet", ticker)
    balancesheets["balancesheet"].insert(0, balancesheet["balancesheet"][0])
    if not("balancesheet" in balancesheets.keys()) or len(balancesheets["balancesheet"]) == 0:
        return []
    balancesheetsData = balancesheets["balancesheet"]
    bookValue = np.array([])
    for bs in balancesheetsData:
        treasuryStock = 0.0
        if bs["treasuryStock"] is not None:
            treasuryStock = bs["treasuryStock"]
           
        bookValue = np.append(bookValue, bs["shareholderEquity"]  / (bs["commonStock"] - treasuryStock))
    return bookValue


In [18]:
#COMPOUND ANNUAL GROWTH RATES (CAGR)
def CAGR(item):
    periods = len(item)
    if periods > 1 :
        first = item[-1]
        last = item[0] 
        cage = ((last/first)**(1/periods))-1
        return cage
    else:
        return 0

In [24]:
def BookValueGrowth_DividendValuation(ticker):    
    # Sharesoutstanding
    shares_outstanding = sharesOutstanding(ticker)
    
    #Book Value
    bookValue = bookValues(ticker) 

    # Current Year Book Value 
    currentYearBookValue = 0
    if len(bookValue) >0:
        currentYearBookValue = bookValue[0]
    
    # Book value growth rate
    bookValueGrowthRate = CAGR(bookValue)

    # Future Book Value
    futureBookValue = futureValue(currentYearBookValue, len(bookValue), bookValueGrowthRate)
    
    # Annual Dividend
    annualDividend = avgDividends(ticker)
    
    # Dicount Rate 10y fed note 
    dicountRate10YearFedNote = 0.68
    
    #B14*((1-(1/(1+B15)^10))/B15)+(B21/(1+B15)^10))
    return annualDividend*((1-(1/(1+ dicountRate10YearFedNote)**10))/dicountRate10YearFedNote)+(futureBookValue/(1+dicountRate10YearFedNote)**10)
 

In [34]:
tickers = ['HTZ']
dataSet = [] 
for ticker in tickers:
    data = {}  
    data["Book Value Dividend"] = BookValueGrowth_DividendValuation(ticker)
    dataSet.append(data.copy())
pd.DataFrame(dataSet) 

[]


,Book Value Dividend
0,0.026222


In [8]:
IntrinsicValue('AAPL')

In [9]:
dict = {'name':["aparna", "pankaj", "sudhir", "Geeku"], 
        'degree': ["MBA", "BCA", "M.Tech", "MBA"], 
        'score':[90, 40, 80, 98]} 

In [10]:
pd.DataFrame(dict) 

,name,degree,score
0,aparna,MBA,90
1,pankaj,BCA,40
2,sudhir,M.Tech,80
3,Geeku,MBA,98
